In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import os
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base

# --- CONFIGURATION ---
# Use a raw string (r"...") to handle Windows paths correctly.
JSON_TRANSCRIPT_PATH = r"C:\Users\apran\Videos\Cin\LIBRARY\Meeting Agent\transcript.json"
MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"

In [ ]:
# --- DATABASE SETUP ---
DATABASE_URL = "sqlite:///tasks.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class Task(Base):
    __tablename__ = "tasks"
    id = Column(Integer, primary_key=True, index=True)
    description = Column(String, index=True)
    assignee = Column(String)
    due_date_str = Column(String)
    status = Column(String, default="To Do")

# Create the table in the database
Base.metadata.create_all(bind=engine)
print("Database and 'tasks' table are ready.")

In [ ]:
def load_transcript_from_json(file_path):
    """Loads and formats the transcript from your JSON file."""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        formatted_transcript = f"Meeting Title: {data['meeting_title']}\n"
        formatted_transcript += f"Date: {data['date']}\n"
        formatted_transcript += f"Participants: {', '.join(data['participants'])}\n\n"
        
        for entry in data['transcript']:
            formatted_transcript += f"{entry['speaker']} ({entry['timestamp']}): {entry['dialogue']}\n"
            
        return formatted_transcript
    except Exception as e:
        print(f"Error loading transcript: {e}")
        return None

def load_model():
    """Loads the tokenizer and model from Hugging Face."""
    print("Loading AI model... (This can take a minute)")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("✅ Model loaded successfully.")
    return tokenizer, model

In [ ]:
def process_transcript(tokenizer, model, transcript):
    """Generates MoM and extracts tasks using the LLM."""
    prompt = f"""
    You are an expert meeting assistant. Analyze the following transcript.
    Your tasks are:
    1. Generate a concise "Minutes of Meeting" summary.
    2. Extract all action items into a structured JSON format.

    The JSON output must be a single object with a key "tasks", which is a list of objects.
    Each task object must have three keys: "task_description", "assignee", and "due_date".

    Respond with a single JSON object containing "minutes" and "tasks".

    Transcript:
    ---
    {transcript}
    ---
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant that processes meeting transcripts."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    try:
        json_response = json.loads(response[response.find('{'):response.rfind('}')+1])
        return json_response
    except json.JSONDecodeError:
        print("Error: Could not decode JSON from the model's response.")
        return None

def save_tasks_to_db(tasks):
    """Saves the extracted tasks into the SQLite database."""
    db = SessionLocal()
    try:
        # Clear existing tasks for this run to avoid duplicates
        db.query(Task).delete()
        db.commit()
        
        for task_item in tasks:
            new_task = Task(
                description=task_item.get("task_description"),
                assignee=task_item.get("assignee"),
                due_date_str=task_item.get("due_date"),
            )
            db.add(new_task)
        db.commit()
        print(f"\n✅ Successfully saved {len(tasks)} tasks to the database.")
    finally:
        db.close()

In [ ]:
# Load the transcript from the JSON file
meeting_transcript = load_transcript_from_json(JSON_TRANSCRIPT_PATH)

if meeting_transcript:
    # Load the model
    tokenizer, model = load_model()
    
    # Process the transcript
    print("\nProcessing transcript with AI...")
    results = process_transcript(tokenizer, model, meeting_transcript)
    
    if results:
        # Display MoM and extracted tasks
        print("\n--- Minutes of Meeting ---")
        print(results.get("minutes", "No summary generated."))
        
        tasks = results.get("tasks", [])
        if tasks:
            print("\n--- Extracted Action Items ---")
            for task in tasks:
                print(f"- {task}")
            # Save tasks to the database
            save_tasks_to_db(tasks)
        else:
            print("\nNo action items were extracted.")

In [ ]:
# --- Agent Functions (Tools) ---
def get_all_tasks():
    """Retrieves all tasks from the database."""
    db = SessionLocal()
    tasks = db.query(Task).all()
    db.close()
    if not tasks:
        return "No tasks found in the database."
    response = "Here are the current tasks:\n"
    for task in tasks:
        response += f"- Task: '{task.description}', Assignee: {task.assignee}, Due: {task.due_date_str}, Status: {task.status}\n"
    return response

def update_task_status(task_description, new_status):
    """Updates the status of a specific task."""
    db = SessionLocal()
    # Use .ilike() for case-insensitive matching
    task = db.query(Task).filter(Task.description.ilike(f"%{task_description}%")).first()
    if task:
        task.status = new_status
        db.commit()
        response = f"✅ Updated status for task '{task.description}' to '{new_status}'."
    else:
        response = f"❌ Could not find a task matching '{task_description}'."
    db.close()
    return response

# --- Interactive Agent Loop ---
print("\n--- 🤖 Task Agent is ready! ---")
print("You can now ask about your tasks. Type 'quit' to exit.")

while True:
    query = input("> ").lower().strip()
    if query == 'quit':
        break
    
    # Simple rule-based logic to call the correct tool
    if query.startswith("update status for"):
        # Example: "update status for 'budget forecast' to 'In Progress'"
        try:
            desc = query.split("'")[1]
            status = query.split("'")[3]
            print(update_task_status(desc, status))
        except IndexError:
            print("Please use the format: update status for 'task description' to 'new status'")
    elif query in ["what are the tasks", "show all tasks", "list tasks"]:
        print(get_all_tasks())
    else:
        print("Sorry, I can't do that. Try 'show all tasks' or 'update status for ... to ...'.")